In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import BernoulliRBM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [2]:
df = pd.read_csv(r"C:\Users\rosem\Downloads\Paper_works\CSVs\Feature_engg_Paper2.csv")
df = df[df['Region'].isin(['CAL','TEX'])]
df

,Date,Hour,Demand,Net_Generation,Total_Interchange,Forecasted_Demand,Coal_Gen,Gas_Gen,Nuclear_Gen,Hydro_Gen,...,Rolling_Mean_24H,Rolling_Mean_3H_Emission,Rolling_Mean_24H_Emission,Total_Gen,Renewable_Pct,Fossil_Pct,Renewable_Demand,Fossil_Demand,Demand_diff1,Demand_diff24
0,2018-07-01 17:00:00,12.5,28976.042,21138.304,-6528.677,28386.220,8.764,7725.951,2261.073,3089.741,...,42604.509,27096.582,17297.042,17521.203,42.950,44.145,1244530.613,1279144.078,89.458,172.875
1,2018-07-01 18:00:00,12.5,29065.500,21217.885,-6554.521,28486.010,8.792,7761.917,2261.062,3110.594,...,41847.195,8244.812,17052.778,17577.073,42.927,44.209,1247693.394,1284966.639,89.458,172.875
2,2018-07-01 19:00:00,12.5,29154.958,21297.467,-6580.365,28585.800,8.819,7797.882,2261.052,3131.446,...,45629.697,24266.186,19327.269,17632.943,42.904,44.273,1250856.313,1290788.823,89.458,172.875
3,2018-07-01 20:00:00,12.5,29244.417,21377.049,-6606.208,28685.590,8.847,7833.847,2261.042,3152.299,...,48609.446,6436.826,20904.933,17688.812,42.881,44.337,1254019.366,1296610.635,89.458,172.875
4,2018-07-01 21:00:00,12.5,29333.875,21456.630,-6632.052,28785.380,8.875,7869.812,2261.031,3173.151,...,49142.567,6326.084,21205.564,17744.682,42.858,44.400,1257182.553,1302432.078,89.458,172.875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525055,2023-06-29 20:00:00,12.5,64967.778,65025.979,58.042,65732.083,8989.486,27552.910,4647.181,32.896,...,49409.849,30361.664,19267.173,46077.069,10.607,79.307,689127.079,5152407.218,-65.642,-1360.549
525056,2023-06-29 21:00:00,12.5,64902.135,64963.474,61.177,65699.302,8993.729,27593.401,4647.292,33.224,...,46137.266,13730.128,18243.586,46106.208,10.566,79.354,685784.748,5150245.412,-65.642,-1414.266
525057,2023-06-29 22:00:00,12.5,64836.493,64900.969,64.312,65666.521,8997.972,27633.892,4647.403,33.552,...,45454.091,12961.713,16905.630,46135.347,10.526,79.401,682451.108,5148073.607,-65.642,-1467.983
525058,2023-06-29 23:00:00,12.5,64770.851,64838.464,67.448,65633.740,9002.215,27674.384,4647.514,33.880,...,39779.804,15939.369,14414.641,46164.486,10.485,79.448,679126.143,5145891.821,-65.642,-1521.700


In [3]:
df.columns.to_list()

['Date',
 'Hour',
 'Demand',
 'Net_Generation',
 'Total_Interchange',
 'Forecasted_Demand',
 'Coal_Gen',
 'Gas_Gen',
 'Nuclear_Gen',
 'Hydro_Gen',
 'Solar_Gen',
 'CO2_Factor_Coal',
 'CO2_Factor_Gas',
 'CO2_Emissions_Coal',
 'CO2_Emissions_Gas',
 'CO2_Total_Emissions',
 'CO2_Intensity_Generated',
 'CO2_Intensity_Consumed',
 'Month',
 'DayOfWeek',
 'Is_Weekend',
 'DayOfYear',
 'WeekOfYear',
 'Region',
 'Season_Autumn',
 'Season_Spring',
 'Season_Summer',
 'Season_Winter',
 'Demand_Prev_Hour',
 'Demand_Yesterday_Same_Hour',
 'Demand_Last_Week_Same_Hour',
 'Emission_Prev_Hour',
 'Emission_Yesterday_Same_Hour',
 'Emission_Last_Week_Same_Hour',
 'Rolling_Mean_3H',
 'Rolling_Mean_24H',
 'Rolling_Mean_3H_Emission',
 'Rolling_Mean_24H_Emission',
 'Total_Gen',
 'Renewable_Pct',
 'Fossil_Pct',
 'Renewable_Demand',
 'Fossil_Demand',
 'Demand_diff1',
 'Demand_diff24']

In [4]:
df['Region'].unique()

array(['CAL', 'TEX'], dtype=object)

# 128

In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import BernoulliRBM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-6))) * 100

feature_cols = [
    'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
    'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
    'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
    'Emission_Prev_Hour', 'Emission_Yesterday_Same_Hour', 'Emission_Last_Week_Same_Hour',
    'Rolling_Mean_3H', 'Rolling_Mean_24H', 'Rolling_Mean_3H_Emission', 'Rolling_Mean_24H_Emission',
    'Total_Gen', 'Renewable_Pct', 'Fossil_Pct', 'Renewable_Demand', 'Fossil_Demand', 'Demand_diff1', 'Demand_diff24'
]
target_cols = ['Demand', 'CO2_Total_Emissions']

# ----------- CHANGE THIS FOR THE REGION ----------
region_code = 'CAL'  # use 'TEX' for Texas

region_df = df[df['Region'] == region_code].copy()
region_df['Date'] = pd.to_datetime(region_df['Date'])
region_df.set_index('Date', inplace=True)

features = region_df[feature_cols]
targets = region_df[target_cols]

scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)

rbm = BernoulliRBM(n_components=20, learning_rate=0.05, n_iter=20, random_state=42)
X_rbm = rbm.fit_transform(X_scaled)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(targets)

def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_rbm, y_scaled, lookback=30)
split = int(0.8 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

results = []

for epoch in [10]:
    model = Sequential([
        LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(y_train.shape[1])
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=0)

    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    results.append({
        'Region': 'California' if region_code == 'CAL' else 'Texas',
        'Epoch': 10,
        'MAE_Demand': mae(y_true[:, 0], y_pred[:, 0]),
        'RMSE_Demand': rmse(y_true[:, 0], y_pred[:, 0]),
        'MAPE_Demand(%)': mape(y_true[:, 0], y_pred[:, 0]),
        'MAE_CO2': mae(y_true[:, 1], y_pred[:, 1]),
        'RMSE_CO2': rmse(y_true[:, 1], y_pred[:, 1]),
        'MAPE_CO2(%)': mape(y_true[:, 1], y_pred[:, 1])
    })

metrics_df = pd.DataFrame(results)
print(metrics_df)


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


274/274 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


274/274 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


274/274 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


274/274 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
       Region  Epoch   MAE_Demand  RMSE_Demand  MAPE_Demand(%)      MAE_CO2  \
0  California      1  2447.765922  3544.383677        7.584357  1376.091984   
1  California      2  2416.889526  3378.013559        7.524468  1117.503281   
2  California      5  1620.907079  2420.234595        5.027921   882.466979   
3  California     10  1571.743445  2338.557317        4.959041   807.433462   

      RMSE_CO2  MAPE_CO2(%)  
0  1693.675481    25.080429  
1  1431.600943    22.471495  
2  1194.391929    16.122040  
3  1122.128822    14.879197  


In [12]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import BernoulliRBM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-6))) * 100

feature_cols = [
    'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
    'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
    'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
    'Emission_Prev_Hour', 'Emission_Yesterday_Same_Hour', 'Emission_Last_Week_Same_Hour',
    'Rolling_Mean_3H', 'Rolling_Mean_24H', 'Rolling_Mean_3H_Emission', 'Rolling_Mean_24H_Emission',
    'Total_Gen', 'Renewable_Pct', 'Fossil_Pct', 'Renewable_Demand', 'Fossil_Demand', 'Demand_diff1', 'Demand_diff24'
]
target_cols = ['Demand', 'CO2_Total_Emissions']

# ----------- CHANGE THIS FOR THE REGION ----------
region_code = 'CAL'  # use 'TEX' for Texas

region_df = df[df['Region'] == region_code].copy()
region_df['Date'] = pd.to_datetime(region_df['Date'])
region_df.set_index('Date', inplace=True)

features = region_df[feature_cols]
targets = region_df[target_cols]

scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)

rbm = BernoulliRBM(n_components=20, learning_rate=0.05, n_iter=20, random_state=42)
X_rbm = rbm.fit_transform(X_scaled)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(targets)

def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_rbm, y_scaled, lookback=30)
split = int(0.75 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

results = []

for epoch in [1, 2, 5, 10]:
    model = Sequential([
        LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(y_train.shape[1])
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=epoch, batch_size=64, verbose=0)

    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    results.append({
        'Region': 'California' if region_code == 'CAL' else 'Texas',
        'Epoch': epoch,
        'MAE_Demand': mae(y_true[:, 0], y_pred[:, 0]),
        'RMSE_Demand': rmse(y_true[:, 0], y_pred[:, 0]),
        'MAPE_Demand(%)': mape(y_true[:, 0], y_pred[:, 0]),
        'MAE_CO2': mae(y_true[:, 1], y_pred[:, 1]),
        'RMSE_CO2': rmse(y_true[:, 1], y_pred[:, 1]),
        'MAPE_CO2(%)': mape(y_true[:, 1], y_pred[:, 1])
    })

metrics_df = pd.DataFrame(results)
print(metrics_df)


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


342/342 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


342/342 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


342/342 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


342/342 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step
       Region  Epoch   MAE_Demand  RMSE_Demand  MAPE_Demand(%)      MAE_CO2  \
0  California      1  2369.253659  3429.353685        7.605353  1172.100240   
1  California      2  2313.837244  3222.024363        7.484562  1094.595168   
2  California      5  1907.054271  2784.085962        6.081959   962.810151   
3  California     10  1720.829641  2501.347141        5.414414   782.051609   

      RMSE_CO2  MAPE_CO2(%)  
0  1534.572971    26.139200  
1  1414.802641    23.362476  
2  1247.506515    20.113968  
3  1103.753238    14.703192  


In [13]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import BernoulliRBM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-6))) * 100

feature_cols = [
    'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
    'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
    'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
    'Emission_Prev_Hour', 'Emission_Yesterday_Same_Hour', 'Emission_Last_Week_Same_Hour',
    'Rolling_Mean_3H', 'Rolling_Mean_24H', 'Rolling_Mean_3H_Emission', 'Rolling_Mean_24H_Emission',
    'Total_Gen', 'Renewable_Pct', 'Fossil_Pct', 'Renewable_Demand', 'Fossil_Demand', 'Demand_diff1', 'Demand_diff24'
]
target_cols = ['Demand', 'CO2_Total_Emissions']

# ----------- CHANGE THIS FOR THE REGION ----------
region_code = 'CAL'  # use 'TEX' for Texas

region_df = df[df['Region'] == region_code].copy()
region_df['Date'] = pd.to_datetime(region_df['Date'])
region_df.set_index('Date', inplace=True)

features = region_df[feature_cols]
targets = region_df[target_cols]

scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)

rbm = BernoulliRBM(n_components=20, learning_rate=0.05, n_iter=20, random_state=42)
X_rbm = rbm.fit_transform(X_scaled)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(targets)

def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_rbm, y_scaled, lookback=30)
split = int(0.7 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

results = []

for epoch in [1, 2, 5, 10]:
    model = Sequential([
        LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(y_train.shape[1])
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=epoch, batch_size=64, verbose=0)

    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    results.append({
        'Region': 'California' if region_code == 'CAL' else 'Texas',
        'Epoch': epoch,
        'MAE_Demand': mae(y_true[:, 0], y_pred[:, 0]),
        'RMSE_Demand': rmse(y_true[:, 0], y_pred[:, 0]),
        'MAPE_Demand(%)': mape(y_true[:, 0], y_pred[:, 0]),
        'MAE_CO2': mae(y_true[:, 1], y_pred[:, 1]),
        'RMSE_CO2': rmse(y_true[:, 1], y_pred[:, 1]),
        'MAPE_CO2(%)': mape(y_true[:, 1], y_pred[:, 1])
    })

metrics_df = pd.DataFrame(results)
print(metrics_df)


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


411/411 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


411/411 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


411/411 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


411/411 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step
       Region  Epoch   MAE_Demand  RMSE_Demand  MAPE_Demand(%)      MAE_CO2  \
0  California      1  2274.429207  3337.667298        6.964718  1124.501396   
1  California      2  2023.631601  3021.572692        6.512377  1057.328273   
2  California      5  1541.558961  2364.758135        4.891674   823.663064   
3  California     10  1508.073114  2206.120854        4.836948   732.601211   

      RMSE_CO2  MAPE_CO2(%)  
0  1462.465955    23.750866  
1  1385.712198    23.905921  
2  1099.612218    17.582415  
3  1017.459664    15.135081  


In [14]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import BernoulliRBM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-6))) * 100

feature_cols = [
    'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
    'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
    'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
    'Emission_Prev_Hour', 'Emission_Yesterday_Same_Hour', 'Emission_Last_Week_Same_Hour',
    'Rolling_Mean_3H', 'Rolling_Mean_24H', 'Rolling_Mean_3H_Emission', 'Rolling_Mean_24H_Emission',
    'Total_Gen', 'Renewable_Pct', 'Fossil_Pct', 'Renewable_Demand', 'Fossil_Demand', 'Demand_diff1', 'Demand_diff24'
]
target_cols = ['Demand', 'CO2_Total_Emissions']

# ----------- CHANGE THIS FOR THE REGION ----------
region_code = 'CAL'  # use 'TEX' for Texas

region_df = df[df['Region'] == region_code].copy()
region_df['Date'] = pd.to_datetime(region_df['Date'])
region_df.set_index('Date', inplace=True)

features = region_df[feature_cols]
targets = region_df[target_cols]

scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)

rbm = BernoulliRBM(n_components=20, learning_rate=0.05, n_iter=20, random_state=42)
X_rbm = rbm.fit_transform(X_scaled)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(targets)

def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_rbm, y_scaled, lookback=30)
split = int(0.65 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

results = []

for epoch in [1, 2, 5, 10]:
    model = Sequential([
        LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(y_train.shape[1])
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=epoch, batch_size=64, verbose=0)

    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    results.append({
        'Region': 'California' if region_code == 'CAL' else 'Texas',
        'Epoch': epoch,
        'MAE_Demand': mae(y_true[:, 0], y_pred[:, 0]),
        'RMSE_Demand': rmse(y_true[:, 0], y_pred[:, 0]),
        'MAPE_Demand(%)': mape(y_true[:, 0], y_pred[:, 0]),
        'MAE_CO2': mae(y_true[:, 1], y_pred[:, 1]),
        'RMSE_CO2': rmse(y_true[:, 1], y_pred[:, 1]),
        'MAPE_CO2(%)': mape(y_true[:, 1], y_pred[:, 1])
    })

metrics_df = pd.DataFrame(results)
print(metrics_df)


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


479/479 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


479/479 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


479/479 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


479/479 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step
       Region  Epoch   MAE_Demand  RMSE_Demand  MAPE_Demand(%)      MAE_CO2  \
0  California      1  2165.071810  3094.221995        7.189979  1031.771014   
1  California      2  1965.686080  2862.166956        6.421459   994.941114   
2  California      5  1663.188571  2427.207410        5.449385   828.371215   
3  California     10  1668.790037  2329.486387        5.348007   797.961104   

      RMSE_CO2  MAPE_CO2(%)  
0  1358.577860    22.411814  
1  1310.277014    21.448885  
2  1107.571530    18.150513  
3  1062.309652    16.048844  


In [18]:
# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.neural_network import BernoulliRBM
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-6))) * 100

feature_cols = [
    'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
    'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
    'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
    'Emission_Prev_Hour', 'Emission_Yesterday_Same_Hour', 'Emission_Last_Week_Same_Hour',
    'Rolling_Mean_3H', 'Rolling_Mean_24H', 'Rolling_Mean_3H_Emission', 'Rolling_Mean_24H_Emission',
    'Total_Gen', 'Renewable_Pct', 'Fossil_Pct', 'Renewable_Demand', 'Fossil_Demand', 'Demand_diff1', 'Demand_diff24'
]
target_cols = ['Demand', 'CO2_Total_Emissions']

# ----------- CHANGE THIS FOR THE REGION ----------
region_code = 'CAL'  # use 'TEX' for Texas

region_df = df[df['Region'] == region_code].copy()
region_df['Date'] = pd.to_datetime(region_df['Date'])
region_df.set_index('Date', inplace=True)

features = region_df[feature_cols]
targets = region_df[target_cols]

scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)

rbm = BernoulliRBM(n_components=20, learning_rate=0.05, n_iter=20, random_state=42)
X_rbm = rbm.fit_transform(X_scaled)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(targets)

def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_rbm, y_scaled, lookback=30)
split = int(0.6 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

results = []

for epoch in [1, 2, 5, 10]:
    model = Sequential([
        LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(y_train.shape[1])
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=epoch, batch_size=64, verbose=0)

    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    results.append({
        'Region': 'California' if region_code == 'CAL' else 'Texas',
        'Epoch': epoch,
        'MAE_Demand': mae(y_true[:, 0], y_pred[:, 0]),
        'RMSE_Demand': rmse(y_true[:, 0], y_pred[:, 0]),
        'MAPE_Demand(%)': mape(y_true[:, 0], y_pred[:, 0]),
        'MAE_CO2': mae(y_true[:, 1], y_pred[:, 1]),
        'RMSE_CO2': rmse(y_true[:, 1], y_pred[:, 1]),
        'MAPE_CO2(%)': mape(y_true[:, 1], y_pred[:, 1])
    })

metrics_df = pd.DataFrame(results)
print(metrics_df)


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


547/547 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


547/547 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


547/547 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


547/547 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step
       Region  Epoch   MAE_Demand  RMSE_Demand  MAPE_Demand(%)      MAE_CO2  \
0  California      1  2469.770853  3383.327542        7.523948  1151.594961   
1  California      2  2039.973275  2970.757015        6.346266  1181.774445   
2  California      5  1820.743429  2576.128207        5.819877  1003.583629   
3  California     10  1692.549581  2401.905880        5.390564   884.673039   

      RMSE_CO2  MAPE_CO2(%)  
0  1497.455486    23.759856  
1  1518.708226    24.030683  
2  1291.457196    20.154654  
3  1173.026325    17.574456  


# TEX

In [20]:
# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.neural_network import BernoulliRBM
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-6))) * 100

feature_cols = [
    'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
    'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
    'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
    'Emission_Prev_Hour', 'Emission_Yesterday_Same_Hour', 'Emission_Last_Week_Same_Hour',
    'Rolling_Mean_3H', 'Rolling_Mean_24H', 'Rolling_Mean_3H_Emission', 'Rolling_Mean_24H_Emission',
    'Total_Gen', 'Renewable_Pct', 'Fossil_Pct', 'Renewable_Demand', 'Fossil_Demand', 'Demand_diff1', 'Demand_diff24'
]
target_cols = ['Demand', 'CO2_Total_Emissions']

# ----------- CHANGE THIS FOR THE REGION ----------
region_code = 'TEX'  # use 'TEX' for Texas

region_df = df[df['Region'] == region_code].copy()
region_df['Date'] = pd.to_datetime(region_df['Date'])
region_df.set_index('Date', inplace=True)

features = region_df[feature_cols]
targets = region_df[target_cols]

scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)

rbm = BernoulliRBM(n_components=20, learning_rate=0.05, n_iter=20, random_state=42)
X_rbm = rbm.fit_transform(X_scaled)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(targets)

def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_rbm, y_scaled, lookback=30)
split = int(0.8 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

results = []

for epoch in [1, 2, 5, 10]:
    model = Sequential([
        LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(y_train.shape[1])
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=epoch, batch_size=64, verbose=0)

    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    results.append({
        'Region': 'California' if region_code == 'CAL' else 'Texas',
        'Epoch': epoch,
        'MAE_Demand': mae(y_true[:, 0], y_pred[:, 0]),
        'RMSE_Demand': rmse(y_true[:, 0], y_pred[:, 0]),
        'MAPE_Demand(%)': mape(y_true[:, 0], y_pred[:, 0]),
        'MAE_CO2': mae(y_true[:, 1], y_pred[:, 1]),
        'RMSE_CO2': rmse(y_true[:, 1], y_pred[:, 1]),
        'MAPE_CO2(%)': mape(y_true[:, 1], y_pred[:, 1])
    })

metrics_df = pd.DataFrame(results)
print(metrics_df)


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


274/274 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


274/274 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


274/274 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


274/274 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
  Region  Epoch   MAE_Demand  RMSE_Demand  MAPE_Demand(%)      MAE_CO2  \
0  Texas      1  3093.460069  3975.387541        6.071697  2276.677470   
1  Texas      2  3427.983785  4607.059350        6.529870  2374.360113   
2  Texas      5  3471.817897  4587.428921        6.673322  2217.381688   
3  Texas     10  3291.616915  4313.330335        6.335581  1937.815075   

      RMSE_CO2  MAPE_CO2(%)  
0  2920.864071    16.809765  
1  3029.577746    17.709836  
2  2802.145750    16.372327  
3  2439.561808    14.138975  


In [21]:
# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.neural_network import BernoulliRBM
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-6))) * 100

feature_cols = [
    'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
    'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
    'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
    'Emission_Prev_Hour', 'Emission_Yesterday_Same_Hour', 'Emission_Last_Week_Same_Hour',
    'Rolling_Mean_3H', 'Rolling_Mean_24H', 'Rolling_Mean_3H_Emission', 'Rolling_Mean_24H_Emission',
    'Total_Gen', 'Renewable_Pct', 'Fossil_Pct', 'Renewable_Demand', 'Fossil_Demand', 'Demand_diff1', 'Demand_diff24'
]
target_cols = ['Demand', 'CO2_Total_Emissions']

# ----------- CHANGE THIS FOR THE REGION ----------
region_code = 'TEX'  # use 'TEX' for Texas

region_df = df[df['Region'] == region_code].copy()
region_df['Date'] = pd.to_datetime(region_df['Date'])
region_df.set_index('Date', inplace=True)

features = region_df[feature_cols]
targets = region_df[target_cols]

scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)

rbm = BernoulliRBM(n_components=20, learning_rate=0.05, n_iter=20, random_state=42)
X_rbm = rbm.fit_transform(X_scaled)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(targets)

def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_rbm, y_scaled, lookback=30)
split = int(0.75 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

results = []

for epoch in [1, 2, 5, 10]:
    model = Sequential([
        LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(y_train.shape[1])
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=epoch, batch_size=64, verbose=0)

    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    results.append({
        'Region': 'California' if region_code == 'CAL' else 'Texas',
        'Epoch': epoch,
        'MAE_Demand': mae(y_true[:, 0], y_pred[:, 0]),
        'RMSE_Demand': rmse(y_true[:, 0], y_pred[:, 0]),
        'MAPE_Demand(%)': mape(y_true[:, 0], y_pred[:, 0]),
        'MAE_CO2': mae(y_true[:, 1], y_pred[:, 1]),
        'RMSE_CO2': rmse(y_true[:, 1], y_pred[:, 1]),
        'MAPE_CO2(%)': mape(y_true[:, 1], y_pred[:, 1])
    })

metrics_df = pd.DataFrame(results)
print(metrics_df)


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


342/342 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


342/342 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


342/342 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


342/342 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
  Region  Epoch   MAE_Demand  RMSE_Demand  MAPE_Demand(%)      MAE_CO2  \
0  Texas      1  4804.370366  6055.743930        9.157279  2366.802933   
1  Texas      2  4522.063300  5516.503283        8.786210  2174.898737   
2  Texas      5  4294.986380  5307.835984        8.340329  2138.056989   
3  Texas     10  3824.289020  4769.463049        7.370641  1897.668101   

      RMSE_CO2  MAPE_CO2(%)  
0  2993.742234    17.114939  
1  2723.842279    15.265899  
2  2811.396432    15.637129  
3  2418.630672    13.456452  


In [22]:
# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.neural_network import BernoulliRBM
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-6))) * 100

feature_cols = [
    'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
    'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
    'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
    'Emission_Prev_Hour', 'Emission_Yesterday_Same_Hour', 'Emission_Last_Week_Same_Hour',
    'Rolling_Mean_3H', 'Rolling_Mean_24H', 'Rolling_Mean_3H_Emission', 'Rolling_Mean_24H_Emission',
    'Total_Gen', 'Renewable_Pct', 'Fossil_Pct', 'Renewable_Demand', 'Fossil_Demand', 'Demand_diff1', 'Demand_diff24'
]
target_cols = ['Demand', 'CO2_Total_Emissions']

# ----------- CHANGE THIS FOR THE REGION ----------
region_code = 'TEX'  # use 'TEX' for Texas

region_df = df[df['Region'] == region_code].copy()
region_df['Date'] = pd.to_datetime(region_df['Date'])
region_df.set_index('Date', inplace=True)

features = region_df[feature_cols]
targets = region_df[target_cols]

scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)

rbm = BernoulliRBM(n_components=20, learning_rate=0.05, n_iter=20, random_state=42)
X_rbm = rbm.fit_transform(X_scaled)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(targets)

def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_rbm, y_scaled, lookback=30)
split = int(0.7 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

results = []

for epoch in [1, 2, 5, 10]:
    model = Sequential([
        LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(y_train.shape[1])
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=epoch, batch_size=64, verbose=0)

    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    results.append({
        'Region': 'California' if region_code == 'CAL' else 'Texas',
        'Epoch': epoch,
        'MAE_Demand': mae(y_true[:, 0], y_pred[:, 0]),
        'RMSE_Demand': rmse(y_true[:, 0], y_pred[:, 0]),
        'MAPE_Demand(%)': mape(y_true[:, 0], y_pred[:, 0]),
        'MAE_CO2': mae(y_true[:, 1], y_pred[:, 1]),
        'RMSE_CO2': rmse(y_true[:, 1], y_pred[:, 1]),
        'MAPE_CO2(%)': mape(y_true[:, 1], y_pred[:, 1])
    })

metrics_df = pd.DataFrame(results)
print(metrics_df)


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


411/411 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


411/411 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


411/411 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


411/411 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step
  Region  Epoch   MAE_Demand  RMSE_Demand  MAPE_Demand(%)      MAE_CO2  \
0  Texas      1  4499.663106  5708.200351        8.758956  2002.153754   
1  Texas      2  4406.041504  5430.424848        8.765065  1904.663526   
2  Texas      5  4024.303178  4987.052072        7.991234  3099.456791   
3  Texas     10  4041.314288  4944.127124        8.056258  2089.103409   

      RMSE_CO2  MAPE_CO2(%)  
0  2656.329423    14.677207  
1  2519.189370    13.483957  
2  3788.247876    22.410937  
3  2871.061295    15.047056  


In [23]:
# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.neural_network import BernoulliRBM
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-6))) * 100

feature_cols = [
    'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
    'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
    'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
    'Emission_Prev_Hour', 'Emission_Yesterday_Same_Hour', 'Emission_Last_Week_Same_Hour',
    'Rolling_Mean_3H', 'Rolling_Mean_24H', 'Rolling_Mean_3H_Emission', 'Rolling_Mean_24H_Emission',
    'Total_Gen', 'Renewable_Pct', 'Fossil_Pct', 'Renewable_Demand', 'Fossil_Demand', 'Demand_diff1', 'Demand_diff24'
]
target_cols = ['Demand', 'CO2_Total_Emissions']

# ----------- CHANGE THIS FOR THE REGION ----------
region_code = 'TEX'  # use 'TEX' for Texas

region_df = df[df['Region'] == region_code].copy()
region_df['Date'] = pd.to_datetime(region_df['Date'])
region_df.set_index('Date', inplace=True)

features = region_df[feature_cols]
targets = region_df[target_cols]

scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)

rbm = BernoulliRBM(n_components=20, learning_rate=0.05, n_iter=20, random_state=42)
X_rbm = rbm.fit_transform(X_scaled)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(targets)

def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_rbm, y_scaled, lookback=30)
split = int(0.65 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

results = []

for epoch in [1, 2, 5, 10]:
    model = Sequential([
        LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(y_train.shape[1])
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=epoch, batch_size=64, verbose=0)

    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    results.append({
        'Region': 'California' if region_code == 'CAL' else 'Texas',
        'Epoch': epoch,
        'MAE_Demand': mae(y_true[:, 0], y_pred[:, 0]),
        'RMSE_Demand': rmse(y_true[:, 0], y_pred[:, 0]),
        'MAPE_Demand(%)': mape(y_true[:, 0], y_pred[:, 0]),
        'MAE_CO2': mae(y_true[:, 1], y_pred[:, 1]),
        'RMSE_CO2': rmse(y_true[:, 1], y_pred[:, 1]),
        'MAPE_CO2(%)': mape(y_true[:, 1], y_pred[:, 1])
    })

metrics_df = pd.DataFrame(results)
print(metrics_df)


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


479/479 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


479/479 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


479/479 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


479/479 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step
  Region  Epoch   MAE_Demand  RMSE_Demand  MAPE_Demand(%)      MAE_CO2  \
0  Texas      1  3558.501255  4598.295861        7.155953  2659.749933   
1  Texas      2  3475.276297  4492.519304        7.033273  3278.222368   
2  Texas      5  4048.213346  5042.899630        8.204922  2615.834322   
3  Texas     10  3867.234739  4822.782158        7.811876  2434.231490   

      RMSE_CO2  MAPE_CO2(%)  
0  3324.068911    19.980534  
1  3994.439054    24.626919  
2  3366.295052    18.853567  
3  3062.518214    17.458389  


In [24]:
# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.neural_network import BernoulliRBM
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense

def mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred) ** 2))
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), 1e-6))) * 100

feature_cols = [
    'Hour', 'Month', 'DayOfWeek', 'Is_Weekend', 'DayOfYear', 'WeekOfYear',
    'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
    'Demand_Prev_Hour', 'Demand_Yesterday_Same_Hour', 'Demand_Last_Week_Same_Hour',
    'Emission_Prev_Hour', 'Emission_Yesterday_Same_Hour', 'Emission_Last_Week_Same_Hour',
    'Rolling_Mean_3H', 'Rolling_Mean_24H', 'Rolling_Mean_3H_Emission', 'Rolling_Mean_24H_Emission',
    'Total_Gen', 'Renewable_Pct', 'Fossil_Pct', 'Renewable_Demand', 'Fossil_Demand', 'Demand_diff1', 'Demand_diff24'
]
target_cols = ['Demand', 'CO2_Total_Emissions']

# ----------- CHANGE THIS FOR THE REGION ----------
region_code = 'TEX'  # use 'TEX' for Texas

region_df = df[df['Region'] == region_code].copy()
region_df['Date'] = pd.to_datetime(region_df['Date'])
region_df.set_index('Date', inplace=True)

features = region_df[feature_cols]
targets = region_df[target_cols]

scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(features)

rbm = BernoulliRBM(n_components=20, learning_rate=0.05, n_iter=20, random_state=42)
X_rbm = rbm.fit_transform(X_scaled)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(targets)

def create_sequences(X, y, lookback=30):
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i-lookback:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_rbm, y_scaled, lookback=30)
split = int(0.6 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

results = []

for epoch in [1, 2, 5, 10]:
    model = Sequential([
        LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(y_train.shape[1])
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=epoch, batch_size=64, verbose=0)

    y_pred_scaled = model.predict(X_test)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test)

    results.append({
        'Region': 'California' if region_code == 'CAL' else 'Texas',
        'Epoch': epoch,
        'MAE_Demand': mae(y_true[:, 0], y_pred[:, 0]),
        'RMSE_Demand': rmse(y_true[:, 0], y_pred[:, 0]),
        'MAPE_Demand(%)': mape(y_true[:, 0], y_pred[:, 0]),
        'MAE_CO2': mae(y_true[:, 1], y_pred[:, 1]),
        'RMSE_CO2': rmse(y_true[:, 1], y_pred[:, 1]),
        'MAPE_CO2(%)': mape(y_true[:, 1], y_pred[:, 1])
    })

metrics_df = pd.DataFrame(results)
print(metrics_df)


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


547/547 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


547/547 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


547/547 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step


C:\Users\rosem\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


547/547 ━━━━━━━━━━━━━━━━━━━━ 17s 30ms/step
  Region  Epoch   MAE_Demand  RMSE_Demand  MAPE_Demand(%)      MAE_CO2  \
0  Texas      1  3798.381270  5001.417314        7.487992  2277.620260   
1  Texas      2  4116.649573  5209.796156        8.207347  2532.555740   
2  Texas      5  4028.111021  5118.288938        7.987473  2516.018724   
3  Texas     10  3700.464917  4552.169511        7.527494  2508.222197   

      RMSE_CO2  MAPE_CO2(%)  
0  2947.546986    16.052985  
1  3163.169569    17.875209  
2  3154.663522    17.233541  
3  3143.743165    17.217024  
